# Assertion Generation for Existing Questions

This notebook demonstrates how to generate assertions for existing data-local and data-global questions that were previously generated without assertions (e.g., when `max_assertions=0` was used or assertions were disabled during question generation).

This is useful when you want to retroactively add assertion-based evaluation capabilities to existing question sets.

In [13]:
# Copyright (c) 2025 Microsoft Corporation.

import sys

sys.path.insert(1, "../../../")

In [14]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [15]:
import json
import logging
import os

from dataclasses import asdict
from pydantic import SecretStr

from benchmark_qed.autoq.io.question import (
    load_questions,
    save_questions,
)
from benchmark_qed.config.llm_config import LLMConfig, LLMProvider
from benchmark_qed.llm.factory import ModelFactory

logging.basicConfig(level=logging.INFO)

if logging.getLogger("httpx") is not None:
    logging.getLogger("httpx").setLevel(logging.ERROR)

## Configuration

In [16]:
# DATA CONFIGS
OUTPUT_QUESTIONS_PATH = "../../output/AP_news/legacy_questions"

# MODEL CONFIGS
API_KEY = SecretStr(os.getenv("OPENAI_API_KEY", ""))
LLM_MODEL = "gpt-4.1"
LLM_PARAMS = {
    "temperature": 0.0,
    "seed": 42,
}  # adjust this based on your model. For example, some reasoning models do not support temperature settings
CONCURRENT_REQUESTS = (
    8  # Control for request concurrency. Adjust this based on your model capacity.
)

llm = ModelFactory.create_chat_model(
    model_config=LLMConfig(
        model=LLM_MODEL,
        api_key=API_KEY,
        llm_provider=LLMProvider.OpenAIChat,
        call_args=LLM_PARAMS,
    )
)

## Generate Assertions for Existing Data-Local Questions

Generate assertions for data-local questions that were previously created without assertions.

In [17]:
from benchmark_qed.autoq.question_gen.data_questions.assertion_gen.local_claim_assertion_gen import (
    LocalClaimAssertionGenerator,
)

# Load existing data-local questions from disk
# Replace with your actual path to existing questions
existing_local_questions = load_questions(
    f"{OUTPUT_QUESTIONS_PATH}/data_local_questions/selected_questions.json"
)

print(f"Loaded {len(existing_local_questions)} existing data-local questions")

# Initialize local assertion generator
local_assertion_generator = LocalClaimAssertionGenerator(
    llm=llm,
    max_assertions=10,  # Maximum number of assertions per question
)

# Generate assertions for each question
updated_local_questions = []
for question in existing_local_questions:
    # Check if question has claims to generate assertions from
    if not (hasattr(question, 'attributes') and question.attributes and 'claims' in question.attributes):
        print(f"Question {question.id} has no claims, skipping assertion generation...")
        updated_local_questions.append(question)
        continue
    
    try:
        # Generate assertions from the question's claims
        claims = question.attributes['claims']
        assertion_result = await local_assertion_generator.agenerate_assertions(
            question_text=question.text,
            claims=claims
        )
        
        # Update question with generated assertions
        if not question.attributes:
            question.attributes = {}
        question.attributes['assertions'] = [asdict(assertion) for assertion in assertion_result.assertions]

        print(f"Generated {len(assertion_result.assertions)} assertions for question: {question.text[:100]}...")
        
    except Exception as e:
        print(f"Failed to generate assertions for question {question.id}: {str(e)}")
    
    updated_local_questions.append(question)

# Save updated questions with assertions
save_questions(
    updated_local_questions,
    f"{OUTPUT_QUESTIONS_PATH}/data_local_questions/",
    "selected_questions_with_assertions",
)

print(f"Saved {len(updated_local_questions)} data-local questions with assertions")

Loaded 50 existing data-local questions
Generated 2 assertions for question: What amendment was made to Article 34 of the French Constitution regarding abortion rights in March ...
Generated 6 assertions for question: What was the Florida Supreme Court's decision regarding the proposed abortion rights amendment for t...
Generated 7 assertions for question: What are the limitations of early prenatal ultrasounds and genetic screenings in detecting fetal abn...
Generated 2 assertions for question: From which countries do Mexican cartels source precursor chemicals for fentanyl production between 2...
Generated 6 assertions for question: Why are junior doctors in South Korea striking in February 2024?...
Generated 9 assertions for question: What are the contributing factors to the rise in flu and COVID-19 infections in the United States du...
Generated 6 assertions for question: What changes to the yellow flag law did Maine Governor Janet Mills propose between January 2024 and ...
Generated

## Generate Assertions for Existing Data-Global Questions

Generate assertions for data-global questions that were previously created without assertions. Global assertion generation uses a map-reduce approach, first generating local assertions from referenced questions, then consolidating them into global assertions.

In [18]:
from benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen import (
    GlobalClaimAssertionGenerator,
)

# Load existing data-global questions from disk
# Replace with your actual path to existing questions
existing_global_questions = load_questions(
    f"{OUTPUT_QUESTIONS_PATH}/data_global_questions/selected_questions.json"
)

print(f"Loaded {len(existing_global_questions)} existing data-global questions")

# Initialize global assertion generator
global_assertion_generator = GlobalClaimAssertionGenerator(
    llm=llm,
    max_assertions=10,  # Maximum number of assertions per question
    batch_size=100,  # Batch size for processing multiple claims
    max_data_tokens=32000, # max input data tokens for the reduce step to generat the final assertions
    concurrent_coroutines=CONCURRENT_REQUESTS,
)

# Generate assertions for each global question
updated_global_questions = []
for question in existing_global_questions:
   
    # Check if question has claims to generate assertions from
    if not (hasattr(question, 'attributes') and question.attributes and 'claims' in question.attributes):
        print(f"Global question {question.id} has no claims, skipping assertion generation...")
        updated_global_questions.append(question)
        continue
    
    try:
        claims = question.attributes['claims']
        
       # Generate assertions using the map-reduce approach
        assertion_result = await global_assertion_generator.agenerate_assertions(
            question_text=question.text,
            claims=claims
        )
        
        # Update question with generated assertions
        if not question.attributes:
            question.attributes = {}
        question.attributes['assertions'] = [asdict(assertion) for assertion in assertion_result.assertions]

        print(f"Generated {len(assertion_result.assertions)} assertions for global question: {question.text[:100]}...")
        
    except Exception as e:
        print(f"Failed to generate assertions for global question {question.id}: {str(e)}")
    
    updated_global_questions.append(question)

# Save updated questions with assertions
save_questions(
    updated_global_questions,
    f"{OUTPUT_QUESTIONS_PATH}/data_global_questions/",
    "selected_questions_with_assertions",
)

print(f"Saved {len(updated_global_questions)} data-global questions with assertions")

INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP CONTEXT: Created 11 batches from 1062 simple claims
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP RESPONSES: Processing 11 batches in parallel


Loaded 50 existing data-global questions


INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP RESPONSES: Successfully processed 11 out of 11 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Merging 196 assertions from 11 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Ranked 196 unique assertions by score and source count
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Selected 196 of 196 assertions within 6520 tokens (limit: 32000)
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE RESPONSE: Consolidating 196 assertions to 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:Successfully consolidated 196 assertions into 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.gl

Generated 10 assertions for global question: Across the dataset, what are the key legislative and policy changes impacting healthcare access and ...


INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP RESPONSES: Successfully processed 9 out of 9 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Merging 146 assertions from 9 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Ranked 146 unique assertions by score and source count
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Selected 146 of 146 assertions within 5172 tokens (limit: 32000)
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE RESPONSE: Consolidating 146 assertions to 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:Successfully consolidated 146 assertions into 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.globa

...

Generated 6 assertions for global question: Across the dataset, what are the key strategies and benefits of implementing shade equity plans in v...


INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP RESPONSES: Successfully processed 1 out of 1 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Merging 10 assertions from 1 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Ranked 10 unique assertions by score and source count
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Selected 10 of 10 assertions within 333 tokens (limit: 32000)
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE RESPONSE: Consolidating 10 assertions to 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:Successfully consolidated 10 assertions into 6
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_

Generated 6 assertions for global question: Across the dataset, what are the key factors influencing sleep duration and quality?...


INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:MAP RESPONSES: Successfully processed 1 out of 1 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Merging 16 assertions from 1 batches
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Ranked 16 unique assertions by score and source count
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE CONTEXT: Selected 16 of 16 assertions within 534 tokens (limit: 32000)
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:REDUCE RESPONSE: Consolidating 16 assertions to 10
INFO:benchmark_qed.autoq.question_gen.data_questions.assertion_gen.global_claim_assertion_gen:Successfully consolidated 16 assertions into 8


Generated 8 assertions for global question: Across the dataset, what are the key strategies and challenges in improving electronic waste recycli...
Saved 50 data-global questions with assertions


## Notes on Assertion Generation

**When to use this approach:**
- You have existing questions that were generated with `max_assertions=0` or without assertion generation
- You want to add evaluation capabilities to previously generated question sets
- You need to regenerate assertions with different parameters or improved prompts

**Input Requirements:**
- Questions must have `claims` in their `attributes` field
- For data-local questions: claims should be a list of claim dictionaries
- For data-global questions: claims can be in various formats (simple or complex)

**Output Format:**
- Assertions are added to the question's `attributes.assertions` field
- Each assertion contains a `statement` that can be used for evaluation
- Questions without valid claims are left unchanged

**Configuration Options:**
- `max_assertions`: Maximum number of assertions to generate per question (default: 10)
- `batch_size`: For global questions, controls how many claims are processed together
- `concurrent_coroutines`: Controls parallel processing for global questions

**Error Handling:**
- Questions without claims are skipped with a warning message
- Individual failures are logged but don't stop the overall process